In [1]:
%pwd

'/home/larry/tensorflow_prac'

In [2]:
import numpy as np
import tensorflow as tf
import time

import utils

In [3]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

In [4]:
# Step 1: Read in data
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


In [5]:
type(train)

tuple

In [6]:
len(train)

2

In [20]:
ary1=train[0]

In [23]:
(ary1[0,:]).shape

(784,)

In [10]:
type(train[0])

numpy.ndarray

In [11]:
type(train[1])

numpy.ndarray

In [14]:
train[0].shape

(55000, 784)

In [15]:
train[1].shape

(55000, 10)

In [16]:
train_data = tf.data.Dataset.from_tensor_slices(train)

In [24]:
iterator = train_data.make_one_shot_iterator()
X, Y = iterator.get_next()

In [28]:
with tf.Session() as sess:
    print(sess.run([X,Y]))
    print(sess.run([X,Y]))

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    